## pre-req
DROP DATABASE IF EXISTS crawl_data;  # delte (crawl_data) database if exists /**/<br>
CREATE DATABASE crawl_data [DEFAULT CHARACTER SET utf8]; # create database (db name : crawl_data, as an example) <br>
CREATE USER crawl_usr IDENTIFIED BY 'test001'; # create user with password (example user name : crawl_usr, password : test001)<br>
GRANT ALL ON crawl_data.* TO crawl_usr; # grant full access to the user

* 참고
https://hyun-am-coding.tistory.com/entry/%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-DB%EC%97%90-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0

1. cmd 열기
2. env를 base가 아닌 새로 만들어서 한다면 사용할 환경을 지정
3. pip install python-dotenv
4. cd 프로젝트하는_파일이_있는_폴더_위치 (예: cd C:/users/project)
5. echo. > .env
cmd는 끝!
5. 프로젝트를 하는 파일이 있는 폴더에 .env 파일이 있을거예요
6. 그 파일 VS Code나 메모장으로 열어서 데이터 베이스에 접속하는데 필요한 정보 입력
    - 예, wanted_envs={'user':'crawl_usr', 'passwd':'test001', 'host':'localhost', 'db' : 'crawl_data'}
7. 그후에 다음 ipynb파일에 나와 있는 코드를 돌리면 접속 정보를 외부에 노출하지 않고 이용하여 정보를 받아올 수 있습니다!

In [1]:
import MySQLdb

In [7]:
from dotenv import load_dotenv
load_dotenv('.env')
import os
from ast import literal_eval

DB_keys = literal_eval(os.getenv('wanted_envs'))
DB_keys

{'user': 'crawl_usr',
 'passwd': 'test001',
 'host': 'localhost',
 'db': 'crawl_data'}

In [8]:
# 방법 1 -  접속 정보를 dict로 저장했다면
conn = MySQLdb.connect(
    user = DB_keys['user'],
    passwd = DB_keys['passwd'],
    host = DB_keys['host'],
    db = DB_keys['db']
)
print(type(conn))

<class 'MySQLdb.connections.Connection'>


In [ ]:
# 방법 2 - 접속 정보를 env에 사용하지 않고
conn = MySQLdb.connect(
    user = 'crawl_usr',
    passwd = 'test001',
    host = 'localhost',
    db = 'crawl_data'
)
print(type(conn))

cursor = conn.cursor()
print(type(cursor))

cursor.execute("CREATE TABLE books (title text, url text)")
conn.commit()

In [3]:
bookname = '처음 시작하는 파이썬'
url = 'www.wikibook.co.kr'
cursor.execute(f'INSERT INTO books VALUES(\"{bookname}\", \"{url}\")')
conn.commit()

#### 실행할때마다 다른 값이 나오지 않게 테이블을 제거하고 다시 생성하는 코드

In [4]:
import MySQLdb
conn = MySQLdb.connect(
    user = 'crawl_usr',
    passwd = 'test001',
    host = 'localhost',
    db = 'crawl_data'
)
# 커서 생성하기
cursor = conn.cursor()

# 실행할 때마다 다른 값이 나오지 않게 테이블을 제거해두기
cursor.execute('DROP TABLE IF EXISTS books')
# 테이블을 생성하기
cursor.execute("CREATE TABLE books (title text, url text)")

# 데이터 저장하기
bookname = '처음 시작하는 파이썬'
url = 'www.wikibook.co.kr'
cursor.execute(f'INSERT INTO books VALUES(\"{bookname}\", \"{url}\")')

# 커밋하기
conn.commit()

# 연결 종료 
conn.close()

### 크롤링 DB에 넣기 - 멜론에서 상위 100개 노래

In [5]:
import MySQLdb
import requests
from bs4 import BeautifulSoup

In [13]:
if __name__ == '__main__':
    RANK = 100 
    header = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
    req = requests.get('http://www.melon.com/chart/week/index.htm', headers = header)
    html = req.text
    parse = BeautifulSoup(html, 'html.parser')
    
    titles = parse.find_all('div', {'class' : 'ellipsis rank01'})
    singers = parse.find_all('div', {'class' : 'ellipsis rank02'})
    
    title = []
    singer = []
    
    for t in titles:
        title.append(t.find('a').text)
    
    for s in singers:
        singer.append(s.find('span', {'class' : 'checkEllipsis'}).text)
    items = [item for item in zip(title, singer)]

conn = MySQLdb.connect(
    user = 'crawl_usr',
    passwd = 'test001',
    host = 'localhost',
    db = 'crawl_data')

cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS melon')
cursor.execute("CREATE TABLE melon (`rank` int, title text, singer text)")
i = 1

for item in items:
    cursor.execute(
        f'INSERT INTO melon VALUES({i},\"{item[0]}\", \"{item[1]}\")')
    i += 1

conn.commit()
conn.close()

***

### Reading data from a MySQL table usig python

In [9]:
# establishing the connection
conn = MySQLdb.connect(
    user = 'crawl_usr',
    passwd = 'test001',
    host = 'localhost',
    db = 'crawl_data')

# Creating a cursor object
cursor = conn.cursor()

# Retrieving data 
sql = '''SELECT * from melon''' 

# execute the query
cursor.execute(sql)

# Fetching 1st row from the table
result = cursor.fetchone()
print(result)

# Fetching all
results = cursor.fetchall()
print(results)

# closing the connection
conn.close()

100